# Document Similarity using LDA and gensim

Context-based similar documents problem can help us to find the similar documents of document which are necceesary. Many times, people do recommend the several items, which could be related to several data that customers read. Several techniques are there to find the similarity between the document but, in this project we will be using the concept of LDA and gensim.

In [40]:
from gensim import corpora
import codecs
from gensim import models, similarities
import os
import pickle
import codecs
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas


In [43]:

colnames=["id","title","content"]
inf = pandas.read_csv('articles1.csv',columns=colnames, nrows=10)
#data= inf.content.tolist()
#fileid=inf.id.tolist()
#name=inf.title.tolist()


# What is the Structure of Dataset?


In [44]:
inf.shape

(50000, 10)

Our data contains 5000 entries and 10 columns. Following is the glimpse of our dataset.

In [28]:
inf.head()

,,,,,,,id,title,content
NaN,id,title,publication,author,date,year,month,url,content
0.0,17283,House Republicans Fret About Winning Their Health Care Suit - The New York Times,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1.0,17284,Rift Between Officers and Residents as Killings Persist in South Bronx - The New York Times,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2.0,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial Bias, Dies at 106 - The New York Times",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3.0,17286,"Among Deaths in 2016, a Heavy Toll in Pop Music - The New York Times",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."


# What is/are some useful features in the dataset?

our basic aim is to find any similar documents to our query. so for here in our dataset, there are four main coloumns, which are really neccessary for us to build our project. Four main coloums are id, title, content,url.

# What are Features in dataset, that will be neccessary to do investigation into the features?

Our main features would be 
ID
Title
Content

In [29]:
stopwords_file="stopwords.txt"
stopwords=[]
if stopwords_file is not None:
    sreader = codecs.open("stopwords.txt", 'r', 'utf8')
    for line in sreader.readlines():
        stopwords.append(line.replace('\n', ''))
stoplist = set(stopwords)
sentences = [[word for word in content.split(u' ') if word not in stoplist] for content in data]

dictionary=corpora.Dictionary(sentences)
dictionary.save("dictionary.dict")
corpus=[dictionary.doc2bow(sentence) for sentence in sentences]
corpora.MmCorpus.serialize("corpus.mm",corpus)

A next step is to create a Bag of Words on the dataset. A dictionary would be created from all the processed documents. A dictionary will contain the number of times word appeared in the training set.   

# DATA Pre-Processing

Several steps are invloved in data preprocessing.

Tokenization: in this step, we split all content of document into sentences and that into words.
all the stop words are removed.


Using the data collected from the dataset, we will be preprocessing it and make, it more clean. since there are several documents, in each document, we need to tokenize it into sentences and words, also removing all the stop words. once the data is cleaned we will make a dictionary. and then finally a corpus. A corpus is the collection of the input document.

In [30]:
model = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=100)
model.save("model.lda")
model = models.ldamodel.LdaModel.load("model.lda")
word_occurs=False
high_frequency_words=15

# What is LDA Algorithm?

LDA or Latent Dirichlet Allocation is another topic model, and it does the classification of several douments according to their niche. LDA builds a document model, and model of words in each topic. All of this is modeled as the Direchlet Distributions.

In [41]:
if word_occurs:
    for i in range(model.num_topics):
        print('Title ', i+1, ': ', model.print_topic(i, topn=high_frequency_words))
else:
    for i in range(model.num_topics):
        hfwords = [a[0] for a in model.show_topic(i, topn=high_frequency_words)]
        collection = 'Title' + str(i+1) + ':'
        for word in hfwords:
            collection += ' ' + word
        print(collection)

In [39]:
dictionary = corpora.Dictionary.load('dictionary.dict')
corpus = corpora.MmCorpus("corpus.mm")
lda = models.LdaModel.load("model.lda")

index = similarities.MatrixSimilarity(lda[corpus])
index.save("simIndex.index")




# How does Similarity class work?

A main class is similarity, which is used to build the index of the collection of documents. once the index is build,we can check how similar is the query document to the all the vectors of numbers. 

Simailarity Class divides the index into several small sub indices. with the use of other functionalities, such as MatrixSimilary we can find the similarity between the documents.

# Gensim doc2bow

we have used the library Gensim Doc2bow, generally it creates a dectionary and it gives the number of words appeared. and it is saved in the query_bow.

# How to test a Model on Query?

In [ ]:
docname = "Query.txt"
doc = open(docname, 'r').read()
query_bow = dictionary.doc2bow(doc.lower().split())
vec_lda = lda[query_bow]

sims = index[vec_lda]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print("Document ID" + " Similarity-Score")
for i in range(10):
    print("Document ID : " + str(sims[i][0])+"\n"+"content : "+data[i]+"\n"+"content_similarity : "+str(sims[i][1]))'''

# Summary

Intuitively, the results are ways much better and quite accuarate. Such code can be used to build the recommendation system, like with the help of such data, we can recommend people about blogs and artiicles according their context they like to read.